In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np


In [ ]:
# Import the Fast Food dataset, drop missing data
fast_food_data = pd.read_csv('resources/fast_food_data_us.csv')
fast_food_data = fast_food_data.dropna()
fast_food_data.head()


In [ ]:
# Import the Obesity dataset, drop missing data

obesity_data = pd.read_csv('resources/obesity_data_state_2020.csv')
obesity_data = obesity_data.dropna()
obesity_data.head()


## We need to convert the states to state codes in order to work with them - possible solution https://pypi.org/project/us/

In [ ]:
### Data Cleanup 
#connect to Census API - Julia 
#convert Census API zip code into state

#figure out census data - class info



In [ ]:
#merge tables by state 

#create data frame including state, fast food chain, obesity rate

#data frame population and state
